# 2D Engine

In [ ]:
import sys
sys.path.append("..")

from importlib import reload
import src.engine
reload(src.engine)

from src.config import FluidSimConfig2D, FluidSimConfig3D, FluidSpawnGroup
from src.engine import FluidSimulationEngine

# Seed
import random
random.seed(0)
import numpy as np
np.random.seed(0)

BOUNDS = 60.0

config = FluidSimConfig2D()
config.simulation_time = 35.0
config.time_step = 0.01 # 3000 Time steps
config.domain_size = [BOUNDS, BOUNDS]
config.kernel_radius = 1.0
config.rest_density = 1
config.pressure_multiplier = 200
config.viscosity_multiplier = 3.0
config.spawn_groups= [
    FluidSpawnGroup(
        num_particles=1000,
        spawn_time=0.0,
        spawn_position=[BOUNDS / 4, BOUNDS / 2],
        spawn_bounds=[BOUNDS / 2, BOUNDS],
        velocity=[0.0, 0.0],
        spawn_pattern="uniform_random"
    ),
    FluidSpawnGroup(
        num_particles=1000,
        spawn_time=10.0,
        spawn_position=[BOUNDS / 4, BOUNDS / 2],
        spawn_bounds=[BOUNDS / 2, BOUNDS],
        velocity=[0.0, 0.0],
        spawn_pattern="uniform_random"
    )
]

engine = FluidSimulationEngine(config)

_ = engine.step()

In [ ]:
import matplotlib.pyplot as plt
import os

if not os.path.exists("2d/output"):
    os.makedirs("2d/output")

i = 0
frame = 0
while True:
    if engine.is_running:
        positions, velocities, pressure_forces, viscosity_forces = engine.step()
    else:
        break

    if i % 10 == 0:
        fig, ax = plt.subplots()

        # Draw density
        density_map = engine.get_density_map(resolution=64)
        ax.imshow(density_map, cmap='hot', origin='lower', extent=[0, BOUNDS, 0, BOUNDS], vmin=0, vmax=config.rest_density * 2)

        # Draw particles
        ax.scatter(positions[:, 0], positions[:, 1], s=1)

        # # Draw arrows to symbolize the velocities and forces
        # for pos, vel in zip(positions, velocities):
        #     ax.arrow(pos[0], pos[1], vel[0] * 0.1, vel[1] * 0.1, head_width=0.5, head_length=0.5, fc='r', ec='r')
        # for pos, force in zip(positions, pressure_forces):
        #     ax.arrow(pos[0], pos[1], force[0] * 0.1, force[1] * 0.1, head_width=0.5, head_length=0.5, fc='b', ec='b')
        # for pos, force in zip(positions, viscosity_forces):
        #     ax.arrow(pos[0], pos[1], force[0] * 0.1, force[1] * 0.1, head_width=0.5, head_length=0.5, fc='g', ec='g')

        ax.set_xlim(0, BOUNDS)
        ax.set_ylim(0, BOUNDS)
        
        ax.set_title(f"Frame {frame}")
        plt.savefig(f"2d/output/frame_{frame:04d}.png")
        plt.close()
        frame += 1
        print(frame)

    i += 1


In [ ]:
!ffmpeg -framerate 10 -y -i 2d/output/frame_%04d.png -c:v libx264 -pix_fmt yuv420p 2d/output/video.mp4

# 3D Engine

In [ ]:
import sys
sys.path.append("..")

from importlib import reload
import src.engine
reload(src.engine)

from src.config import FluidSimConfig2D, FluidSimConfig3D, FluidSpawnGroup
from src.engine import FluidSimulationEngine

BOUNDS = 60.0

config = FluidSimConfig3D()
config.simulation_time = 100.0
config.time_step = 0.1
config.domain_size = [BOUNDS, BOUNDS, BOUNDS]
config.viscosity_multiplier = 7.0
config.kernel_radius = 1.5
config.rest_density = 0.8
config.pressure_multiplier = 25
config.spawn_groups= [
    FluidSpawnGroup(
        num_particles=20_000,
        spawn_time=0.0,
        spawn_position=[BOUNDS / 8, BOUNDS / 2, BOUNDS / 2],
        spawn_bounds=[BOUNDS / 4, BOUNDS, BOUNDS],
        velocity=[0.0, 0.0, 0.0],
        spawn_pattern="uniform_random"
    ),
    FluidSpawnGroup(
        num_particles=20_000,
        spawn_time=25.0,
        spawn_position=[BOUNDS / 8, BOUNDS / 2, BOUNDS / 2],
        spawn_bounds=[BOUNDS / 4, BOUNDS / 2, BOUNDS],
        velocity=[0.0, -5.0, 0.0],
        spawn_pattern="uniform_random"
    )
]

engine = FluidSimulationEngine(config)

_ = engine.step()

In [ ]:
from skimage import measure
import numpy as np

def save_obj(output_path, verts, faces, normals):
    if normals is None:
        # 1. Compute face normals
        #    For each face, take two edges and compute the cross product.
        v0 = verts[faces[:, 1]] - verts[faces[:, 0]]
        v1 = verts[faces[:, 2]] - verts[faces[:, 0]]
        face_normals = np.cross(v0, v1)

        # 2. Accumulate the face normals into per-vertex normals
        normals = np.zeros_like(verts)
        for i, face in enumerate(faces):
            for j in face:
                normals[j] += face_normals[i]

        # 3. Normalize the per-vertex normals
        lengths = np.linalg.norm(normals, axis=1, keepdims=True)
        # Avoid division by zero (possible if some vertices are not in any face)
        lengths[lengths == 0] = 1
        normals /= lengths


    with open(output_path, 'w') as f:
        # OBJ format: write vertices
        for vert in verts:
            f.write(f"v {vert[0]} {vert[1]} {vert[2]}\n")

        # OBJ format: write normals
        for normal in normals:
            f.write(f"vn {normal[0]} {normal[1]} {normal[2]}\n")

        # OBJ format: write faces (OBJ uses 1-based index)
        for face in faces:
            # We'll write the vertex normals as well, which requires the face line to include both vertex and normal indices
            f.write(f"f {face[0]+1}//{face[0]+1} {face[1]+1}//{face[1]+1} {face[2]+1}//{face[2]+1}\n")

    print(f"OBJ file saved as {output_path}")

In [ ]:
import os

if not os.path.exists("3d/output"):
    os.makedirs("3d/output")

In [ ]:
import matplotlib.pyplot as plt

i = 0
frame = 0
while True:
    if engine.is_running:
        positions, _, _, _ = engine.step()
    else:
        break

    i += 1
    if i % 1 == 0:
        densities = engine.get_voxel_representation(resolution=128, blur_sigma=2.0)
        densities = np.array(densities)
        print(i)
        print(densities.max())

        try:
            verts, faces, normals, values = measure.marching_cubes(densities, level=0.05)
        except ValueError:
            print("Marching cubes failed, skipping frame.")
            verts = []
            faces = []
            normals = []
        output_path = f"3d/output/frame_{frame:04d}.obj"
        save_obj(output_path, verts, faces, None)
        frame += 1

In [ ]:
app_made = False if "app" not in locals() else True

if not app_made:
    from PyQt5 import QtWidgets
    from vispy import scene
    from vispy import app
    import sys
    import numpy as np
    app = QtWidgets.QApplication(sys.argv)

In [ ]:
import vispy
import time

class Canvas(scene.SceneCanvas):
    
    def __init__(self, sim):
        scene.SceneCanvas.__init__(self, keys='interactive', size=(512, 512), show=True)
        self.unfreeze()  # Unfreeze to add a new attribute
        self.engine = sim
        self.view = self.central_widget.add_view()
        self.points = scene.visuals.Markers()
        # Transform points so x is up and y is right and z is forward
        positions = np.array(self.engine.positions)[:, [0, 2, 1]]
        self.points.set_data(np.array(positions), edge_color=None, face_color=(0, 0.8, 1, 1), size=5)
        self.view.add(self.points)
        self.view.camera = scene.cameras.TurntableCamera(fov=45, elevation=30, azimuth=30)
        self.view.camera.set_range((0, BOUNDS), (0, BOUNDS), (0, BOUNDS))

        self.has_waited = False
        self.timer = vispy.app.Timer('auto', connect=self.on_timer, start=True)

    def on_timer(self, event):

        if not self.has_waited:
            self.has_waited = True
            time.sleep(1)  # Wait for 1 second before starting the simulation
            return

        # Update the positions and velocities
        self.positions, self.velocities, pressure_forces, viscosity_forces = self.engine.step()
        self.positions = np.array(self.positions)
        # Update the points data
        positions = np.array(self.engine.positions)[:, [0, 2, 1]]
        self.points.set_data(positions, edge_color=None, face_color=(0, 0.8, 1, 1), size=5)
        self.update()  # Repaint the scene

engine = FluidSimulationEngine(config)
canvas = Canvas(engine)
canvas.show()
app.exec_()